In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [10]:

# Extracting bills
bills = [heading.text.strip() for heading in result_headings if heading.text.strip()]

# Function to parse bill information
def parse_bill(bill):
    # Extract bill type (H.R., S., H.J.Res., etc.)
    bill_type = re.search(r'[A-Z.]+', bill).group()
    # Extract the number immediately following "H.R." and before "—"
    number = re.search(r'H\.R\.(\d+)(?=\s*—)', bill).group(1) if re.search(r'H\.R\.(\d+)(?=\s*—)', bill) else None
    
    if bill_type.startswith('H') and number:
        return 'house-bill', 'house', number
    elif bill_type.startswith('S'):
        return 'senate-bill', 'senate', number
    elif bill_type.startswith('H.J.Res'):
        return 'house-joint-resolution', 'house', number
    return None

# Create a DataFrame to hold the parsed bill information
new_data = []
for bill in bills:
    parsed_bill = parse_bill(bill)
    if parsed_bill:
        office, office_type, number = parsed_bill
        new_data.append({'original_name': bill, 'office': office, 'number': number})

new_bills_df = pd.DataFrame(new_data)

In [11]:
new_bills_df

,original_name,office,number
0,H.R.7125 — 118th Congress (2023-2024),house-bill,7125
1,H.R.7124 — 118th Congress (2023-2024),house-bill,7124
2,H.R.7123 — 118th Congress (2023-2024),house-bill,7123
3,H.R.7122 — 118th Congress (2023-2024),house-bill,7122
4,H.R.7121 — 118th Congress (2023-2024),house-bill,7121
...,...,...,...
95,H.R.7030 — 118th Congress (2023-2024),house-bill,7030
96,H.R.7029 — 118th Congress (2023-2024),house-bill,7029
97,H.R.7028 — 118th Congress (2023-2024),house-bill,7028
98,H.R.7027 — 118th Congress (2023-2024),house-bill,7027
